In [5]:
from relaxed.mah import get_an_from_am, get_mah
from relaxed.correlations import (
    add_box_indices,
    get_2d_corr,
    get_opt_corr,
    spearmanr,
    vol_jacknife_err,
)
from relaxed.models import opcam_dict, prepare_datasets, training_suite
import numpy as np


In [9]:
from pathlib import Path
MAH_DIR = Path('../..').joinpath("data", "processed", "bolshoi_m12")

In [4]:
opcam_dict

{'cvir': {'mbin': 0.5, 'order': -1},
 'vmax/vvir': {'mbin': 0.397, 'order': -1},
 't/|u|': {'mbin': 0.67, 'order': 1},
 'x0': {'mbin': 0.738, 'order': 1},
 'q': {'mbin': 0.67, 'order': -1},
 'b_to_a': {'mbin': 0.673, 'order': -1},
 'c_to_a': {'mbin': 0.644, 'order': -1},
 'spin': {'mbin': 0.54, 'order': 1},
 'spin_bullock': {'mbin': 0.54, 'order': 1}}

In [11]:
mah_data = get_mah(MAH_DIR, cutoff_missing=0.05, cutoff_particle=0.05)
cat = mah_data["cat"]
ma_peak = mah_data["ma_peak"]  # for alpha fits and gradients.
am = mah_data["am"]
scales = mah_data["scales"]
mass_bins = mah_data["mass_bins"]
add_box_indices(cat)

# prepare catalog with all a_m
am_names = [f"am_{ii}" for ii in range(len(mass_bins))]
for ii in range(len(mass_bins)):
    cat.add_column(am[:, ii], name=am_names[ii])

In [30]:
params = ('t/|u|',)

mbin = opcam_dict[params[0]]['mbin']
opt_idx = np.where(mass_bins > mbin)[0][0]

rng = np.random.default_rng()
info = {
    "am": {
        "x": am_names,
        "y": params,
    },

    "am_one": {
        "x": [am_names[opt_idx]],
        "y": params,
    },
}
datasets, _, cat_test = prepare_datasets(cat, info, rng)
n_params = len(params)

data = {
    "multicam_am": {
        "xy": datasets["am_one"]["train"],
        "n_features": 1,
        "n_targets": 1,
        "model": "linear",
        "kwargs": {"use_multicam": True},
    },

    "optcam": {
        "xy": datasets["am"]["train"],
        "n_features": 100,
        "n_targets": 1,
        "model": "cam",
        "kwargs": {
            "mass_bins": mass_bins,
            "opt_mbin": opcam_dict[params[0]]["mbin"],
            "cam_order": opcam_dict[params[0]]["order"],
        },
    },
}

models = training_suite(data)


In [31]:
x_test, y_test = datasets['am_one']["test"]
y_pred = models['multicam_am'].predict(x_test)
spearmanr(y_test, y_pred)

0.6120078308294005

In [32]:
x_test, y_test = datasets['am']["test"]
y_pred = models['optcam'].predict(x_test)
spearmanr(y_test, y_pred)

0.6119969013712215